In [ ]:
import visa                             # Import PyVISA library
import time
from struct import *                    # Import Struct module (interpret strings as packed binary data)
import numpy as np
#import VISAresourceExtentions

In [ ]:
# Initialization

rm = visa.ResourceManager()             # Create Resource Manager object
rs = rm.list_resources()                # Method to list the available resources
print(rs[0])                            
counter = rm.open_resource(rs[0])       # Assign the returned object to the instrument variable (i.e. counter)

In [ ]:
# Initial settings

print(counter.query('*IDN?'))           # Query the Identification string 
counter.write('*RST;*CLS')              # Reset the instrument, clear the Error queue
counter.timeout = 1e7                   # Acquisition timeout (ms) - set it higher than the acquisition time
counter.query('SYST:ERR?')              # Error Checking 

In [ ]:
#Basic settings

counter.write("FUNC 'FREQ:BTB 1'")                                 # Setup for frequency back-to-back measurement from channel A (1) - Set up for period back-to-back is "FUNC 'PER:BTB 1'" 
counter.write('CALC:AVER:STAT OFF')                                # Enable/disable statitics; 
counter.write('INP:LEV:AUTO OFF; :INP:LEV 0')                      # Enable/Disable autotrigger on channel A; Trigger level (V)
counter.write('CAL:INT:AUTO OFF; :DISP:ENAB OFF')                  # Enable/Disable reciprocal counter that uses an interpolating technique to increase the resolution (if OFF -> Dead time = 0); Enable/Disable the display (if OFF increase the GPIB speed)
counter.write('FORMAT:TINF ON; :FORMAT PACKED')                    # Read timestamp of each measurement; Redout in ASCII/REAL/PACKED mode - Readout format: ASCII/REAL -> [freq (Hz), tstamp (sec)] ; PACKED -> [freq (Hz), tstamp (picosec)] 
counter.query('SYST:ERR?')                                         # Error Checking 
#time.sleep(1)                                                      # Wait (sec)

In [ ]:
#counter.write('*ESE 0;*SRE 0')                      # To be developed
#counter.wait_on_event(counter, RQS, 10000)
#counter.last_status
#counter.query('*ESR?')
#counter.read()
#counter.query('SYST:ERR?')

In [ ]:
# Fetch frequency/timestamps lists for different Gate times                                                             

start = time.time()

FreqTimeList = []                                           # Define list: list=[] ; Define numpy array:  arr=np.array([])

Taulist = [1e-5 * 10**(n/5) for n in range(20)]             # Define list of gate times (sec) - NotEvenlySpaced: 1e-5 * 2n - EvenlySpaced: 1e-5 * 10**(n/5) - ...
Sampleslist = [10]*20#[int(1e5 * 10**(-n/5)) for n in range(20)]    # Define list of samples to take for each gate time - [10**(5-i) for i in range(4) for x in range(5)] // int(1e5 * 10**(-n/5)) // 1000 // ...

#Taulist = [1e-5, 2e-5, 4e-5, 6e-5, 8e-5, 1e-4, 2e-4, 4e-4, 6e-4, 8e-4, 1e-3, 2e-3, 4e-3, 6e-3, 8e-3, 1e-2, 2e-2, 4e-2, 6e-2, 8e-2]    
#Sampleslist = [50000, 25000, 16667, 10000, 6250, 4000, 2565, 1588, 1011, 637, 402, 254, 160, 101, 64, 41, 26, 16, 11, 7]    

for tau, samples in zip(Taulist, Sampleslist):

    #samples = {tau == 1e-3: 100, tau == 1e-1: 10, tau == 1: 2}.get(True)    # Instead of if(): ... elif(): ... elif() ... 
        
    #if (tau == 4e-5):   break                                               # Break the loop when condition is fulfilled
        
    print(f'Gate time = {tau} s, Samples = {samples}')            
    
    counter.write(f'TRIG:COUNT 1; :ARM:COUNT {samples}')                  # Set number of Samples (maximum = 1e5, for more set x=100) - N samples inside one block (Triggerings); Number of blocks (Armings)
    counter.write(f'SENSE:ACQ:APER {tau}')                                # Set Gate time (sec)
    counter.query('INIT;*OPC?')                                           # Initialize acquisition; *OPC? query waits until the acquisition ends 
    
    x = 1
    if (samples > 1e4): [x, samples] =  [10, samples/10]                  # Condition - Takes into account that maximum number of fetchable samples = 1e4
        
    for i in range(x):
            
        # Read Data in BINARY format (packed)
    
        #counter.write('FETCH:ARR? 1000')                                                              
        #counter.query_binary_values(datatype='s', is_big_endian=True)                                  
        bytesvalues = counter.query_binary_values(f'FETCH:ARR? {samples}', datatype='s', is_big_endian=True)   # Fetch Binary values - Rate = 7,313*10^3 Samples/s 
        freqtimelist = unpack('>'+'dQ'*int(samples), bytesvalues[0])                                           # Convert (unpack) into a readable format - Readout format (tuple) -> (freq [Hz], timestamps [ps], freq, tstamp, ...)
        #freqtimelist = bytesvalues                                                                             # Save list of bytes
        #print(bytesvalues) 
    
        # Read Data in ASCII format
    
        #freqtimelist = counter.query_ascii_values(f'FETCH:ARR? {samples}', converter = 's')  # Fetch and Convert list of ASCII values into a list of strings 
    
        #if  freqtimelist[-1] == '' :    del freqtimelist[-1]                                 # Delete the last element of the list when it is an empty string = ''
        #freqtimelist[-1] = freqtimelist[-1].split('\n')[0]                                   # Delete in the last element of the list of strings every character after the number ( i.e. \n0E..)
        #freqtimelist = [float(i) for i in freqtimelist]                                      # Convert list of strings into float numbers
        #print(freqtimelist)
    
        if (i==0): freqtimeList = freqtimelist                            # Create a bigger List appending all the fetched data lists during the loop     
        else:  freqtimeList = freqtimeList + freqtimelist        
    
        FreqTimeList.append(freqtimeList)                                 # Append a value to the list at each iteration
         
    end = time.time()     
    print(end-start)                                                      # Acquisition time for each Gate time (sec)

end = time.time()
print(end-start)                                                          # Total acquisition time (sec)
            
counter.query('SYST:ERR?')  

In [ ]:
# Write binary string to different files for each Gate time

for tau, samples, freqtimeList in zip(Taulist, Sampleslist, FreqTimeList):
    
    BigBytesString = b''.join(freqtimeList)     # Join the all the bytes string elements of the list into one bytes string

    f = open(f'/home/raffo/Scrivania/AllanDeviation/Counter_VCO/FreqBTB-Tstamps_Tau{tau}s_Samples{samples}_VCO_Binary', 'w+b')
    f.write(BigBytesString)
    f.close()

In [ ]:
# Read binary string from different files for each Gate time

FreqTimeList = []

for tau, samples in zip(Taulist, Sampleslist):
    
    f = open(f'/home/raffo/Scrivania/AllanDeviation/Counter_VCO/FreqBTB-Tstamps_Tau{tau}s_Samples{samples}_VCO_Binary', 'r+b')
    BigBytesString = f.read()

    # Convert (unpack) binary string into a ASCII tuple

    Bytes = len(BigBytesString)                                    # Number of bytes of the binary string
    freqtimeList = unpack('>'+'dQ'*int(Bytes/16), BigBytesString)  # Bytes of the single (freq/tstamp) sample = 16 -> int(Bytes/16) is the total number of samples
    FreqTimeList.append(freqtimeList)

In [ ]:
# Overlapping ADEV calculation

start = time.time()

for tau, samples, freqtimeList in zip(Taulist, Sampleslist, FreqTimeList):

    freqtimearray = np.array(freqtimeList)                     # Convert list to numpy array
    freqarray=(freqtimearray[::2])                             # Take the elements of the array at even steps
    #timearray=(freqtimearray[1::2])                            # Take the elements of the array at odd steps
    
    freqmean = freqarray.mean()                                # Calculate the mean frequency over all the frequency samples
    #print(freqmean)
    
    Adevlist = []                                              # Define list: list=[] ; Define numpy array:  arr=np.array([])
    
    print(f'Gate time = {tau} s, Samples = {samples}') 
                
    deltafreqarray = (freqarray[1:]-freqarray[:-1])**2         # Calculate the square of all the differences of consecutive pairs of frequencies
    #print(len(deltafreqarray))
    adev = (np.mean(deltafreqarray)/2)**(1/2)                  # Calculate Adev
    print(adev)
    #print(adev/freqmean)
        
    Adevlist.append(adev)                                                 # Append a value to the list at each iteration

    #end = time.time()     
    #print(end-start)                                                      # Acquisition time for each Gate time (sec)

end = time.time()
print(end-start)                                                          # Total acquisition time (sec)

In [ ]:
print(Taulist)  
print(Adevlist)
print(Sampleslist)

In [ ]:
# ADEV vs Gate time plot

import matplotlib.pyplot as plt
import pandas as pd

# Convert lists to numpy arrays

TauArray = np.array(Taulist)      
AdevArray = np.array(Adevlist)
SamplesArray = np.array(Sampleslist)

print(repr(SamplesArray))
print(repr(TauArray))
print(repr(AdevArray))

ADEVArray = AdevArray/freqmean   # Calculate normalized Adev

print(repr(ADEVArray))

plt.plot(TauArray, ADEVArray, 'o-')
plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.show()

In [ ]:
counter.write('*RST;*CLS') 

In [ ]:
counter.close()